In [1]:
import numpy as np
import scipy.io
import os
import pandas as pd
#! install pingouin
import pingouin as pg
import logging
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import warnings
warnings.filterwarnings("ignore")

file_path = os.getcwd()
file_path = file_path.split('/')
root_path = '/'.join(file_path[:-2]) + '/'
# root_path = '/home/dayoung/dFC/RL/'
data_path = os.path.join(root_path ,'results','GLM')
os.chdir(data_path)

/home/dayoung/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/dayoung/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [19]:
model_idx = 4
data_type = 'act_state_DS'
tmp_name = data_type + str(format(model_idx, '02')) + '.mat'
tmp_data = scipy.io.loadmat(tmp_name)
params_mass = tmp_data['params_mass']  # (K , Nsubj , EV)

K = params_mass.shape[0]
Nsubj = params_mass.shape[1]
EV_required = 4
EV_start = 2
subj_idx = np.ones((EV_required, ))

for si in range(1, Nsubj):
    tmp = np.ones((EV_required, )) * (si + 1)
    subj_idx = np.concatenate([subj_idx, tmp])  # (Nsubj x EV_required, )

effect_pe_idx = np.tile([1, 1, 2, 2], Nsubj)
effect_outcome_idx = np.tile([1, 0], Nsubj * 2)
feed_type_idx = np.tile([1, 2, 3, 4], Nsubj)

data_whole_k = params_mass[0, :, EV_start:(EV_start + EV_required)].reshape(-1, 1)  # -> (162 * K, 1)
state_idx = np.ones((Nsubj*EV_required,))
for ki in range(K):
    data_k = params_mass[ki, :,EV_start:(EV_start + EV_required)]  # (54, EV_required) <- 4 : pw, pl, nw, nl // 3 : pw, incong, nl
    data_k = data_k.reshape(-1, 1)  # (216, ) :  s1pw, s1pl, s1nw, s1nl, s2pw, ...
    globals()['RM_s' + str(ki + 1) + '_df'] = pd.DataFrame({'Subj': subj_idx, 'Feed_type': feed_type_idx,'beta' : np.squeeze(data_k)})
    globals()['TW_s' + str(ki + 1) + '_df'] = pd.DataFrame({'PE': effect_pe_idx, 'Outcome': effect_outcome_idx, 'beta' : np.squeeze(data_k)})
    globals()['TW_RM_s' + str(ki + 1) + '_df'] = pd.DataFrame({'Subj': subj_idx, 'PE': effect_pe_idx, 'Outcome': effect_outcome_idx, 'beta' : np.squeeze(data_k)})
    if ki > 0:
        data_whole_k = np.concatenate([data_whole_k, data_k])
        state_idx = np.concatenate([state_idx, np.ones((Nsubj*EV_required,))*(ki+1)])

RM_allState_df = pd.DataFrame({'Subj': np.tile(subj_idx, K), 'Feed_type': np.tile(feed_type_idx, K), 'beta' : np.squeeze(data_whole_k), 'state': state_idx})
print(RM_allState_df)
    
TW_RM_allState_df = pd.DataFrame({'Subj': np.tile(subj_idx, K), 'PE': np.tile(effect_pe_idx, K), 'Outcome': np.tile(effect_outcome_idx, K), 'beta' : np.squeeze(data_whole_k), 'state': state_idx})
print(TW_RM_allState_df)

model number to analyze :  46
     Subj  Feed_type    beta  state
0     1.0          1 -0.5878    1.0
1     1.0          2 -0.3776    1.0
2     1.0          3 -0.6101    1.0
3     1.0          4 -0.3359    1.0
4     2.0          1  0.6391    1.0
..    ...        ...     ...    ...
859  53.0          4 -0.2898    4.0
860  54.0          1 -0.6871    4.0
861  54.0          2 -0.7168    4.0
862  54.0          3 -0.7493    4.0
863  54.0          4 -0.6644    4.0

[864 rows x 4 columns]
     Subj  PE  Outcome    beta  state
0     1.0   1        1 -0.5878    1.0
1     1.0   1        0 -0.3776    1.0
2     1.0   2        1 -0.6101    1.0
3     1.0   2        0 -0.3359    1.0
4     2.0   1        1  0.6391    1.0
..    ...  ..      ...     ...    ...
859  53.0   2        0 -0.2898    4.0
860  54.0   1        1 -0.6871    4.0
861  54.0   1        0 -0.7168    4.0
862  54.0   2        1 -0.7493    4.0
863  54.0   2        0 -0.6644    4.0

[864 rows x 5 columns]


In [20]:
for ki in range(K):
    print("######################## State"+str(ki+1)+" #########################")
    print("############## RM-ANOVA ##############")
    aov1 = pg.rm_anova(dv='beta', within=['PE', 'Outcome'], subject='Subj', data=globals()['TW_RM_s' + str(ki + 1) + '_df'])
    print(aov1.round(4))
    print("############## POST-HOC ##############")   
    print("############## X > Y ##############")   
    posthoc_greater = pg.pairwise_tests(dv='beta', within=['PE', 'Outcome'], subject='Subj', alternative = 'greater', padjust='bonf', data=globals()['TW_RM_s' + str(ki + 1) + '_df'])
    print(posthoc_greater.round(4))
    posthoc_greater = pg.pairwise_tests(dv='beta', within=['Outcome', 'PE'], subject='Subj', alternative = 'greater', padjust='bonf', data=globals()['TW_RM_s' + str(ki + 1) + '_df'])
    print(posthoc_greater.round(4))
    print("############## X < Y ##############")   
    posthoc_lesser = pg.pairwise_tests(dv='beta', within=['PE', 'Outcome'], subject='Subj', alternative = 'less', padjust='bonf', data=globals()['TW_RM_s' + str(ki + 1) + '_df'])
    print(posthoc_lesser.round(4))
    posthoc_lesser = pg.pairwise_tests(dv='beta', within=['Outcome', 'PE'], subject='Subj', alternative = 'less', padjust='bonf', data=globals()['TW_RM_s' + str(ki + 1) + '_df'])
    print(posthoc_lesser.round(4))
    print("########################################################################")

print('######################## State x Feed_type #########################')
aov2 = pg.rm_anova(dv='beta', within=["state", "Feed_type"], subject='Subj', data=RM_allState_df)
print(aov2.round(4))
print("############## POST-HOC ##############")
print("############## X > Y ##############")
posthoc_greater = pg.pairwise_tests(dv='beta', within=["state", "Feed_type"], subject='Subj', alternative = 'greater', padjust='bonf', data=RM_allState_df)
print(posthoc_greater.round(4))
print("############## X < Y ##############")   
posthoc_lesser = pg.pairwise_tests(dv='beta', within=["state", "Feed_type"], subject='Subj', alternative = 'less', padjust='bonf', data=RM_allState_df)
print(posthoc_lesser.round(4))

######################## State1 #########################
############## RM-ANOVA ##############


/home/dayoung/anaconda3/lib/python3.9/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  data.groupby(level=1, axis=1, observed=True, group_keys=False)
/home/dayoung/anaconda3/lib/python3.9/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrameGroupBy.diff with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  data.groupby(level=1, axis=1, observed=True, group_keys=False)


         Source      SS  ddof1  ddof2      MS       F   p-unc  p-GG-corr  \
0            PE  0.1831      1     53  0.1831  7.8897  0.0069     0.0069   
1       Outcome  0.1663      1     53  0.1663  3.1082  0.0837     0.0837   
2  PE * Outcome  0.4339      1     53  0.4339  9.2711  0.0036     0.0036   

      ng2  eps  
0  0.0024  1.0  
1  0.0022  1.0  
2  0.0057  1.0  
############## POST-HOC ##############
############## X > Y ##############


/home/dayoung/anaconda3/lib/python3.9/site-packages/pingouin/pairwise.py:593: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stats["Time"].fillna("-", inplace=True)
/home/dayoung/anaconda3/lib/python3.9/site-packages/pingouin/pairwise.py:593: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

       Contrast PE  A  B Paired Parametric       T   dof alternative   p-unc  \
0            PE  -  1  2   True       True  2.8089  53.0     greater  0.0035   
1       Outcome  -  0  1   True       True  1.7630  53.0     greater  0.0418   
2  PE * Outcome  1  0  1   True       True -0.9906  53.0     greater  0.8368   
3  PE * Outcome  2  0  1   True       True  2.8867  53.0     greater  0.0028   

   p-corr p-adjust    BF10  hedges  
0     NaN      NaN  10.004  0.1003  
1     NaN      NaN   1.256  0.0945  
2  1.0000     bonf   0.472 -0.0559  
3  0.0056     bonf   12.03  0.2447  
       Contrast Outcome  A  B Paired Parametric       T   dof alternative  \
0       Outcome       -  0  1   True       True  1.7630  53.0     greater   
1            PE       -  1  2   True       True  2.8089  53.0     greater   
2  Outcome * PE       0  1  2   True       True -0.9982  53.0     greater   
3  Outcome * PE       1  1  2   True       True  3.6930  53.0     greater   

    p-unc  p-corr p-adjust  

...

/home/dayoung/anaconda3/lib/python3.9/site-packages/pingouin/pairwise.py:593: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stats["Time"].fillna("-", inplace=True)
/home/dayoung/anaconda3/lib/python3.9/site-packages/pingouin/pairwise.py:593: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

       Contrast PE  A  B Paired Parametric       T   dof alternative   p-unc  \
0            PE  -  1  2   True       True  2.1977  53.0        less  0.9838   
1       Outcome  -  0  1   True       True  2.9790  53.0        less  0.9978   
2  PE * Outcome  1  0  1   True       True  2.3982  53.0        less  0.9900   
3  PE * Outcome  2  0  1   True       True  2.1561  53.0        less  0.9822   

   p-corr p-adjust   BF10  hedges  
0     NaN      NaN   0.37  0.0703  
1     NaN      NaN  0.066  0.1203  
2     1.0     bonf  0.248  0.0858  
3     1.0     bonf  0.401  0.1513  
       Contrast Outcome  A  B Paired Parametric       T   dof alternative  \
0       Outcome       -  0  1   True       True  2.9790  53.0        less   
1            PE       -  1  2   True       True  2.1977  53.0        less   
2  Outcome * PE       0  1  2   True       True  0.8986  53.0        less   
3  Outcome * PE       1  1  2   True       True  1.7653  53.0        less   

    p-unc  p-corr p-adjust   BF10

/home/dayoung/anaconda3/lib/python3.9/site-packages/pingouin/pairwise.py:593: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stats["Time"].fillna("-", inplace=True)


             Contrast state    A    B Paired Parametric       T   dof  \
0               state     -  1.0  2.0   True       True -2.1793  53.0   
1               state     -  1.0  3.0   True       True -2.9455  53.0   
2               state     -  1.0  4.0   True       True -1.4644  53.0   
3               state     -  2.0  3.0   True       True -0.7195  53.0   
4               state     -  2.0  4.0   True       True  0.7282  53.0   
5               state     -  3.0  4.0   True       True  1.3578  53.0   
6           Feed_type     -    1    2   True       True -1.1527  53.0   
7           Feed_type     -    1    3   True       True -0.6654  53.0   
8           Feed_type     -    1    4   True       True  2.0048  53.0   
9           Feed_type     -    2    3   True       True  0.3364  53.0   
10          Feed_type     -    2    4   True       True  2.4075  53.0   
11          Feed_type     -    3    4   True       True  2.2238  53.0   
12  state * Feed_type   1.0    1    2   True       

/home/dayoung/anaconda3/lib/python3.9/site-packages/pingouin/pairwise.py:593: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stats["Time"].fillna("-", inplace=True)


In [ ]:
# after {performing group_comp.m => generating _AB_DS_...mat file}

mi = 7 # 7~9
region_level = 3
region_level_label = ['node', 'edge', 'state', 'system', 'system_dmn']
data = scipy.io.loadmat(os.path.join(root_path,'data', region_level_label[region_level]+'_AB_DS'+str(mi)+'.mat'))
A = data['A']
B = data['B']
print(A[0].shape)
print(A[0][4].shape)
print(B.shape)

if region_level == 2:
    ROI_name_whole = ['State 1', 'State 2', 'State 3', 'State 4']
elif region_level == 0:
    ROI_name_whole = scipy.io.loadmat(os.path.join(root_path,'data', 'shaefer100_subcortex_7system.mat'))
    ROI_name_whole = ROI_name_whole['cortex_label']
    ROI_name_whole = [ROI_name_whole[i][0][0] for i in range(ROI_name_whole.shape[0])]
    sub_label = ['R-HIP', 'R-AMY', 'R-pTHA', 'R-aTHA', 'R-NAc', 'R-PUT', 'R-CAU',
    'L-HIP', 'L-AMY', 'L-pTHA', 'L-aTHA', 'L-NAc', 'L-PUT', 'L-CAU',
    'L-GPe', 'L-GPi', 'L-SNc', 'L-red', 'L-SNr', 'L-PBPN', 'L-VTA', 'L-VP', 'L-HABN', 'L-hypoTHAL',  'L-MM', 'L-STN',
    'R-GPe', 'R-GPi', 'R-SNc', 'R-red', 'R-SNr', 'R-PBPN', 'R-VTA', 'R-VP', 'R-HABN', 'R-hypoTHAL', 'R-MM', 'R-STN',
    'DR', 'LC', 'LDTg', 'MnR', 'mRt', 'PAG', 'PBC', 'PnO', 'PTg']
    ROI_name_whole = ROI_name_whole + sub_label
elif region_level == 3:    
    net = ['VIS', 'SOM', 'DAN', 'VAN', 'LIM', 'FPN', 'DMN']
    ROI_name_whole = list()
    for ni1 in range(len(net)):
        for ni2 in range(len(net)):
            if ni2 >= ni1:
                ROI_name_whole.append(net[ni1] + '_' + net[ni2])
    print(ROI_name_whole)
elif region_level == 4:    
    net = ['VIS', 'SOM', 'DAN', 'VAN', 'LIM', 'FPN', 'DMN_dm', 'DMN_c', 'DMN_mt']
    ROI_name_whole = list()
    for ni1 in range(len(net)):
        for ni2 in range(len(net)):
            if ni2 >= ni1:
                ROI_name_whole.append(net[ni1] + '_' + net[ni2])
    print(ROI_name_whole)

if np.isin(region_level, [3, 4]):
    Nnet = len(net)
    ir = list()
    ic = list()
    for ii in range(Nnet):
        it = ii
        while it < Nnet:
            ir.append(ii)
            ic.append(it)
            it = it + 1
    annot_template = np.repeat(np.expand_dims(np.tile(np.array(['  ']), Nnet), axis = 1),  Nnet, axis = 1)
    annot_rt = annot_template.copy()
    annot_lt = annot_template.copy()
    right_mat = np.zeros((Nnet, Nnet))
    wrong_mat = np.zeros((Nnet, Nnet))

sig_roi = list()
for roi in range(A[0].shape[0]):
    df_tmp = pd.DataFrame({"beta":A[0][roi].flatten(), "condition":B.flatten(), "Subj": np.repeat(np.linspace(1, 26, 26), 5)})
    wrong_mat[ic[roi], ir[roi]] = np.mean(df_tmp.loc[df_tmp["condition"] == 1]["beta"])  # wrong trials
    right_mat[ic[roi], ir[roi]] = np.mean(df_tmp.loc[df_tmp["condition"] == 0]["beta"])  # right trials
    aov1 = pg.rm_anova(dv='beta', within='condition', subject='Subj', data=df_tmp, correction=True)
    if aov1["p-unc"][0] < 0.05:
        sig_roi.append(roi)
        print("######################## ROI :" + ROI_name_whole[roi]+" #########################")
        print("############## RM-ANOVA ##############")
        print(aov1.round(4))
        print("############## POST-HOC ##############")
        print("############## X > Y ##############")   
        posthoc_greater = pg.pairwise_tests(dv='beta', within='condition', subject='Subj', alternative = 'greater', padjust='bonf', data=df_tmp)
        print(posthoc_greater.round(4))
        print("############## X < Y ##############")
        posthoc_lesser = pg.pairwise_tests(dv='beta', within='condition', subject='Subj', alternative = 'less', padjust='bonf', data=df_tmp)
        print(posthoc_lesser.round(4))
        print("########################################################################")
        if posthoc_greater["p-unc"][0] < 0.05:
            annot_lt[ic[roi], ir[roi]] = '*'
        elif posthoc_lesser["p-unc"][0] < 0.05:
            annot_rt[ic[roi], ir[roi]] = '*'
        print('Lose case mean: ', np.mean(df_tmp.loc[df_tmp["condition"] == 1]["beta"]).round(4))
        print('Win case mean: ', np.mean(df_tmp.loc[df_tmp["condition"] == 0]["beta"]).round(4))

print('number of lose case : ' + str(df_tmp.loc[df_tmp["condition"] == 1]["beta"].shape[0]))
print('number of win case : ' + str(df_tmp.loc[df_tmp["condition"] == 0]["beta"].shape[0]))

if np.isin(region_level, [3, 4]):
    if Nnet > 7:
        figsize = (8, 4.8)
    else:
        figsize = (6, 4.3)
    plot_data_inter = wrong_mat
    diag_tmp = np.eye(Nnet, Nnet) * np.sum(np.tril(plot_data_inter, -1))/np.where(np.tril(plot_data_inter, -1))[0].shape[0]
    plot_data_inter = diag_tmp + np.tril(plot_data_inter, -1) + np.tril(plot_data_inter, -1)
    # Create a mask to separate the diagonal elements
    mask = np.zeros_like(plot_data_inter, dtype=bool)
    mask[np.diag_indices_from(mask)] = True

    plot_data_inter[np.where(np.triu(np.ones(Nnet), 1))] = np.NaN
    # Inter (uper triangle)
    fig_w, ax = plt.subplots(figsize=figsize, constrained_layout=True)  
    sns.heatmap(data=plot_data_inter, mask=mask, cmap="Wistia", annot = False, fmt="", 
                annot_kws={'size':30, 'fontweight':'bold'}, linewidths=.5, cbar=False, ax=ax, 
                xticklabels=net, yticklabels=net, vmin=-0.06, vmax=0.062244)
    # Intra (diagonal)
    plot_data_intra = wrong_mat
    diag_tmp = np.eye(Nnet, Nnet) * np.diag(plot_data_intra)
    plot_data_intra = diag_tmp + (np.ones(Nnet) - np.eye(Nnet, Nnet)) * np.mean(np.diag(plot_data_intra))
    sns.heatmap(data=plot_data_intra, mask=~mask, cmap="Greens", annot = False, fmt="", 
                annot_kws={'size':30, 'fontweight':'bold'}, linewidths=.5, cbar=False, ax=ax, 
                xticklabels=net, yticklabels=net, vmin=-0.0707, vmax=0.02)

    for i in range(Nnet):
        for j in range(Nnet):
            plt.text(j + 0.3, i + 0.85, annot_rt[i, j], {'fontsize':30, 'color':'red', 'weight':'bold'})
            plt.text(j + 0.3, i + 0.85, annot_lt[i, j], {'fontsize':30, 'color':'navy', 'weight':'bold'})   

    cbar1 = ax.figure.colorbar(ax.collections[0], ax=ax, orientation="vertical", pad=0.05)
    cbar2 = ax.figure.colorbar(ax.collections[1], ax=ax, orientation="vertical", pad=0.1)
    #plt.title('Wrong case')
    #
    plot_data_inter = right_mat
    diag_tmp = np.eye(Nnet, Nnet) * np.sum(np.tril(plot_data_inter, -1))/np.where(np.tril(plot_data_inter, -1))[0].shape[0]
    plot_data_inter = diag_tmp + np.tril(plot_data_inter, -1) + np.tril(plot_data_inter, -1)
    # Create a mask to separate the diagonal elements
    mask = np.zeros_like(plot_data_inter, dtype=bool)
    mask[np.diag_indices_from(mask)] = True

    plot_data_inter[np.where(np.triu(np.ones(Nnet), 1))] = np.NaN
    # Inter (uper triangle)
    fig_r, ax = plt.subplots(figsize=figsize, constrained_layout=True)  
    sns.heatmap(data=plot_data_inter, mask=mask, cmap="Wistia", annot = False, fmt="", 
                annot_kws={'size':30, 'fontweight':'bold'}, linewidths=.5, cbar=False, ax=ax, 
                xticklabels=net, yticklabels=net, vmin=-0.06, vmax=0.062244)
    # Intra (diagonal)
    plot_data_intra = wrong_mat
    diag_tmp = np.eye(Nnet, Nnet) * np.diag(plot_data_intra)
    plot_data_intra = diag_tmp + (np.ones(Nnet) - np.eye(Nnet, Nnet)) * np.mean(np.diag(plot_data_intra))
    sns.heatmap(data=plot_data_intra, mask=~mask, cmap="Greens", annot = False, fmt="", 
                annot_kws={'size':30, 'fontweight':'bold'}, linewidths=.5, cbar=False, ax=ax, 
                xticklabels=net, yticklabels=net, vmin=-0.0707, vmax=0.02)

    for i in range(Nnet):
        for j in range(Nnet):
            plt.text(j + 0.3, i + 0.85, annot_lt[i, j], {'fontsize':30, 'color':'red', 'weight':'bold'})
            plt.text(j + 0.3, i + 0.85, annot_rt[i, j], {'fontsize':30, 'color':'navy', 'weight':'bold'})   

    cbar1 = ax.figure.colorbar(ax.collections[0], ax=ax, orientation="vertical", pad=0.05)
    cbar2 = ax.figure.colorbar(ax.collections[1], ax=ax, orientation="vertical", pad=0.1)
    #plt.title('Right case')

    if Nnet > 7:
        pdf_file_path = os.path.join(root_path,'results','Figures','fig6c_DS'+ '{0:02d}'.format(mi) +'_dmn.pdf')
    else:
        pdf_file_path = os.path.join(root_path,'results','Figures','fig6c_DS'+ '{0:02d}'.format(mi) +'.pdf')
    with PdfPages(pdf_file_path) as pdf:
        pdf.savefig(fig_w)
        pdf.savefig(fig_r)